# Lab - Case Based Reasoning (CBR)

Ein (sehr) grundlegendes Labor in dem die Grundzüge des CBL dargestellt werden und das Erstellen einer Ähnlichkeitsfunktion im Vordergrund steht.

Initialisierung und Datenimport

In [11]:
import pandas as pd
import numpy as np

In [12]:
data = pd.read_csv('./data/Versicherung.csv', sep=';')
data.head()

,Geschlecht,Familienstand,Alter,Kinder,Bildungsstand,Beruf,Jahresgehalt,Versicherungstarif
0,weiblich,geschieden,35,1,Bachelor,angestellt,65112,Tarif B
1,maennlich,ledig,27,0,Master,verbeamtet,71322,Tarif B
2,maennlich,verheiratet,57,1,Promotion,angestellt,66340,Tarif C
3,weiblich,verheiratet,71,0,Hauptschule,im Ruhestand,39500,Tarif D
4,weiblich,ledig,64,0,Bachelor,angestellt,59460,Tarif C


Aufgabe 1: Nur Alter und Jahresgehalt

Beide sind numerisch, daher ist die Ähnlichkeitsfunktion einheitlich/homogen

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

def aehnlichkeit(person1, person2):
    """
    Berechnet die Ähnlichkeit zwischen zwei Personen.

    Args:
        person1: Eine Zeile des Datensatzes als Dictionary.
        person2: Eine Zeile des Datensatzes als Dictionary.

    Returns:
        float: Ein Wert zwischen 0 und 1, der die Ähnlichkeit angibt.
    """

    # Merkmale, die für den Vergleich berücksichtigt werden
    merkmale = [ 'Alter', 'Jahresgehalt']

    # Gewichte für die Merkmale (optional, hier gleich gewichtet)
    gewichte = [1] * len(merkmale)

    # Vektoren erstellen, die die Merkmale der Personen repräsentieren
    vektor1 = [person1[merkmal] for merkmal in merkmale]
    vektor2 = [person2[merkmal] for merkmal in merkmale]

    # Für numerische Merkmale: Normalisierung (z.B. Min-Max-Skalierung)
    # ... (Hier könnten Sie eine Normalisierung implementieren, wenn nötig)

    # Für kategoriale Merkmale: Test auf Gleichheit
    # ... (auch explizite Ähnlichkeitswerte zwischen Ausprägungen - Merkmalshierarchie)

    # Cosinus Ähnlichkeit berechnen (hier sollte Sie eine eigene entwerfen)
    aehnlichkeit = cosine_similarity([vektor1], [vektor2])[0][0]

    # Gewichtete Ähnlichkeit (optional)
    # aehnlichkeit = sum(gewichte[i] * aehlichkeit[i] for i in range(len(merkmale))) / sum(gewichte)

    return aehnlichkeit


Test am Beispiel

In [14]:
person_a = data.iloc[0].to_dict()
person_b = data.iloc[1].to_dict()

print(person_a)
print(person_b)

a = aehnlichkeit(person_a, person_b)

print("Ähnlichkeit: ",a)


{'Geschlecht': 'weiblich', 'Familienstand': 'geschieden', 'Alter': 35, 'Kinder': 1, 'Bildungsstand': 'Bachelor', 'Beruf': 'angestellt', 'Jahresgehalt': 65112, 'Versicherungstarif': 'Tarif B'}
{'Geschlecht': 'maennlich', 'Familienstand': 'ledig', 'Alter': 27, 'Kinder': 0, 'Bildungsstand': 'Master', 'Beruf': 'verbeamtet', 'Jahresgehalt': 71322, 'Versicherungstarif': 'Tarif B'}
Ähnlichkeit:  0.9999999873641946


Vergleichsfunktion zur Bestimmung des ähnlichsten Falls

In [15]:
def finde_aehnlichsten_fall(df, person_zu_vergleichen):
    """
    Findet die Person im DataFrame, die die größte Ähnlichkeit zur gegebenen Person hat.

    Args:
        df: Ein Pandas DataFrame mit den Personendaten.
        person_zu_vergleichen: Ein Dictionary, das die Merkmale der Person zum Vergleich enthält.

    Returns:
        tuple: Ein Tupel mit (index, ähnlichkeit), wobei index der Index der ähnlichsten Person im DataFrame und ähnlichkeit der Ähnlichkeitswert ist.
    """

    max_aehnlichkeit = -1
    index_aehlichste_person = -1

    for i, row in df.iterrows():
        person_im_df = row.to_dict()

        a = aehnlichkeit(person_zu_vergleichen, person_im_df)

        if a > max_aehnlichkeit:
            max_aehnlichkeit = a
            index_aehlichste_person = i
            aehnlichster_datensatz = row

    return index_aehlichste_person, max_aehnlichkeit, aehnlichster_datensatz


Test am Beispiel

Person generieren mit Angaben Alter und Jahresgehalt

In [16]:
def erstelle_Person(alter, jahresgehalt):
    """
    Erstellt eine neue Zeile für den DataFrame basierend auf Alter und Jahresgehalt.

    Args:
        alter: Das Alter der Person.
        jahresgehalt: Das Jahresgehalt der Person.

    Returns:
        pd.Series: Eine neue Series (Zeile) für den DataFrame.
    """

    # Definiere die Spaltennamen (passe an, wenn nötig)
    spalten = ['Geschlecht', 'Familienstand', 'Alter', 'Kinder', 'Bildungsstand', 'Beruf', 'Jahresgehalt', 'Versicherungstarif']

    # Erstelle ein Dictionary mit den gegebenen Werten und "null" für die anderen
    neue_zeile = {
        'Geschlecht': 'null',
        'Familienstand': 'null',
        'Alter': alter,
        'Kinder': 'null',
        'Bildungsstand': 'null',
        'Beruf': 'null',
        'Jahresgehalt': jahresgehalt,
        'Versicherungstarif': 'null'
    }
    
    # Konvertiere das Dictionary
    return neue_zeile
    

In [17]:
testperson = erstelle_Person(30, 50000)
print(testperson)

{'Geschlecht': 'null', 'Familienstand': 'null', 'Alter': 30, 'Kinder': 'null', 'Bildungsstand': 'null', 'Beruf': 'null', 'Jahresgehalt': 50000, 'Versicherungstarif': 'null'}


In [18]:
index, aehnlich, eintrag = finde_aehnlichsten_fall(data, testperson)
tarif = eintrag['Versicherungstarif']
print(f"Die ähnlichste Person hat den Index {index} und eine Ähnlichkeit von {aehnlich}.")
print(f"Der Versicherungstarif ist daher {tarif}.")

Die ähnlichste Person hat den Index 92 und eine Ähnlichkeit von 0.9999999999987702.
Der Versicherungstarif ist daher Tarif A.


Erstellen einer Fallbasis (CBL Zyklus)

In [19]:
cases = pd.DataFrame(columns=['Geschlecht', 'Familienstand', 'Alter', 'Kinder', 'Bildungsstand', 'Beruf', 'Jahresgehalt', 'Versicherungstarif'])


Die Fallbasis ist zunächst leer. Diese wird nun nach dem CBL Algorithmus (Instance Based Learning) gefüllt. Vorgehen: Ein Fall wird aufgenommen, wenn er bisher nicht richtig klassifiziert wird.

In [20]:
# Füge den ersten Eintrag von data zu cases hinzu (initialisieren)
initial = data.iloc[0]
cases = cases.append(initial, ignore_index=True)

AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
for index, row in data.iterrows():
    # Finde den ähnlichsten Fall in "cases"
    i, m, case = finde_aehnlichsten_fall(cases, row)

    # Vergleiche den Versicherungstarif
    if case['Versicherungstarif'] != row['Versicherungstarif']:
        # Füge den Datensatz zu "cases" hinzu
        cases = cases.append(row, ignore_index=True)

In [ ]:
print(f"Der Datensatz der Größe {len(data)} wurde in {len(cases)} Fällen gespeichert.")

Der Datensatz der Größe 500 wurde in 215 Fällen gespeichert.


Noch die Qualität prüfen ...

In [ ]:
def treffer(data, cases):
    """
    Überprüft, wie oft finde_aehnlichsten_fall den korrekten Versicherungstarif findet.

    Args:
        data: Der DataFrame, dessen Elemente geprüft werden sollen.
        cases: Der DataFrame, in dem nach ähnlichen Fällen gesucht wird.
        
    Returns:
        int: Anzahl der korrekten Übereinstimmungen.
    """

    korrekte_uebereinstimmungen = 0
    for index, row in data.iterrows():
        _, _, case = finde_aehnlichsten_fall(cases, row)
        if case['Versicherungstarif'] == row['Versicherungstarif']:
            korrekte_uebereinstimmungen += 1
        else:
            print(f"Fehler in Datensatz {index}")
            
    return korrekte_uebereinstimmungen


In [ ]:
anzahl_korrekte_uebereinstimmungen = treffer(data, cases)

print("Anzahl korrekter Übereinstimmungen:", anzahl_korrekte_uebereinstimmungen)

Fehler in Datensatz 11
Fehler in Datensatz 13
Fehler in Datensatz 15
Fehler in Datensatz 19
Fehler in Datensatz 22
Fehler in Datensatz 29
Fehler in Datensatz 31
Fehler in Datensatz 49
Fehler in Datensatz 52
Fehler in Datensatz 64
Fehler in Datensatz 70
Fehler in Datensatz 71
Fehler in Datensatz 77
Fehler in Datensatz 81
Fehler in Datensatz 94
Fehler in Datensatz 99
Fehler in Datensatz 101
Fehler in Datensatz 112
Fehler in Datensatz 115
Fehler in Datensatz 117
Fehler in Datensatz 127
Fehler in Datensatz 152
Fehler in Datensatz 158
Fehler in Datensatz 172
Fehler in Datensatz 173
Fehler in Datensatz 186
Fehler in Datensatz 188
Fehler in Datensatz 192
Fehler in Datensatz 197
Fehler in Datensatz 208
Fehler in Datensatz 211
Fehler in Datensatz 218
Fehler in Datensatz 219
Fehler in Datensatz 224
Fehler in Datensatz 228
Fehler in Datensatz 231
Fehler in Datensatz 235
Fehler in Datensatz 243
Fehler in Datensatz 257
Fehler in Datensatz 260
Fehler in Datensatz 281
Fehler in Datensatz 312
Fehler i